In [1]:
import pandas as pd

docket_df = pd.read_csv('14jan_to_11feb_dockets.csv')

In [2]:
docket_df.head()

,@timestamp,docket_id,docket_rmc,truck_registration,docket_plant_code,docket_datetime_batch_utc,volume_delivered,docket_jobsite_description,detail_docket_consistence_class,detail_docket_consistence_class_min,detail_docket_consistence_class_max,detail_docket_consistence_target,detail_docket_consistence_lower_limit_composite,detail_docket_consistence_upper_limit_composite
0,2025-01-14 07:27:20.000,London-Concrete--41222442,London-Concrete,GN17VMG,BWL,2025-01-14T16:30:00.000Z,7.5,"1 NORTH QUAY, ASPEN WAY",S4,160.0,210.0,0,150,220
1,2025-01-14 07:39:19.000,London-Concrete--41222647,London-Concrete,WU19HYJ,BWL,2025-01-14T07:38:33.089Z,7.5,"1 NORTH QUAY, ASPEN WAY",F5,NaN,NaN,620,570,670
2,2025-01-14 08:09:21.000,London-Concrete--41222647,London-Concrete,WU19HYJ,BWL,2025-01-14T07:49:00.000Z,7.5,"1 NORTH QUAY, ASPEN WAY",F5,NaN,NaN,620,570,670
3,2025-01-14 08:09:24.000,London-Concrete--41222721,London-Concrete,WU19HZV,BWL,2025-01-14T08:09:06.098Z,8.0,TROUBADOR THEATRE - WOOD WHARF,S4,160.0,210.0,0,150,220
4,2025-01-14 08:09:26.000,London-Concrete--41222647,London-Concrete,WU19HYJ,BWL,2025-01-14T07:38:33.089Z,7.5,"1 NORTH QUAY, ASPEN WAY",F5,NaN,NaN,620,570,670


In [14]:
docket_df['docket_datetime_batch_utc'] = pd.to_datetime(docket_df.docket_datetime_batch_utc)
docket_df['docket_update_order_id'] = docket_df.groupby('docket_id')['@timestamp'].rank(method='first', ascending=True)
docket_df_first_docket = docket_df[docket_df['docket_update_order_id'] == 1]
docket_df_further_docket_updates = docket_df[docket_df['docket_update_order_id'] > 1]

In [15]:
docket_df_first_docket = docket_df_first_docket.merge(docket_df_further_docket_updates[
    ['docket_id', '@timestamp', 'docket_datetime_batch_utc']
].rename(columns={'@timestamp': 'docket_update_timestamp', 'docket_datetime_batch_utc': 'docket_datetime_batch_utc_updated'})
, on='docket_id', how='left')

In [16]:
docket_with_updates_df = docket_df_first_docket[docket_df_first_docket.docket_update_timestamp.isnull() == False]
print('proportion of dockets with at least one update: ', len(docket_with_updates_df) / len(docket_df_first_docket))

proportion of dockets with at least one update:  0.9055059523809523


In [21]:
docket_with_updates_df['time_since_first_docket_minutes'] = (pd.to_datetime(docket_with_updates_df['docket_datetime_batch_utc_updated']) - pd.to_datetime(docket_with_updates_df['docket_datetime_batch_utc'])).dt.total_seconds() / 60
docket_with_updates_df = docket_with_updates_df[docket_with_updates_df['time_since_first_docket_minutes'] > 0]

C:\Users\Nicole\AppData\Local\Temp\ipykernel_6296\2268153513.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  docket_with_updates_df['time_since_first_docket_minutes'] = (pd.to_datetime(docket_with_updates_df['docket_datetime_batch_utc_updated']) - pd.to_datetime(docket_with_updates_df['docket_datetime_batch_utc'])).dt.total_seconds() / 60


In [30]:
docket_with_updates_df['update_within_2_hours_flag'] = docket_with_updates_df['time_since_first_docket_minutes'] <= 120
print('proportion of docket updates within 2 hours of first batchtime: ', len(docket_with_updates_df[docket_with_updates_df['update_within_2_hours_flag'] == True]) / len(docket_with_updates_df))

proportion of docket updates within 2 hours of first batchtime:  0.9384816753926701


In [31]:
docket_with_updates_df['update_within_20_minutes_flag'] = docket_with_updates_df['time_since_first_docket_minutes'] <= 20
print('proportion of docket updates within 20 minutes of first batchtime: ', len(docket_with_updates_df[docket_with_updates_df['update_within_20_minutes_flag'] == True]) / len(docket_with_updates_df))

proportion of docket updates within 20 minutes of first batchtime:  0.9319371727748691


In [32]:
docket_with_updates_df['batch_date'] = docket_with_updates_df['docket_datetime_batch_utc'].dt.date
docket_with_updates_df['batch_date_updated'] = docket_with_updates_df['docket_datetime_batch_utc_updated'].dt.date
docket_with_updates_df['same_batch_date_flag'] = docket_with_updates_df['batch_date'] == docket_with_updates_df['batch_date_updated']
docket_with_updates_df['same_batch_date_greater_than_2_hours_flag'] = (docket_with_updates_df['batch_date'] == docket_with_updates_df['batch_date_updated']) & (docket_with_updates_df['time_since_first_docket_minutes'] > 120)
print('proportion of docket updates with the same batch date: ', len(docket_with_updates_df[docket_with_updates_df['same_batch_date_greater_than_2_hours_flag'] == True]) / len(docket_with_updates_df))

proportion of docket updates with the same batch date:  0.0


In [33]:
print('proportion of docket updates with a different batch date: ', len(docket_with_updates_df[docket_with_updates_df['same_batch_date_flag'] == False]) / len(docket_with_updates_df))

proportion of docket updates with a different batch date:  0.061518324607329845


In [38]:
docket_with_updates_df[docket_with_updates_df.same_batch_date_flag == False].sort_values(by='time_since_first_docket_minutes', ascending=False).head(10)

,@timestamp,docket_id,docket_rmc,truck_registration,docket_plant_code,docket_datetime_batch_utc,volume_delivered,docket_jobsite_description,detail_docket_consistence_class,detail_docket_consistence_class_min,...,docket_update_order_id,docket_update_timestamp,docket_datetime_batch_utc_updated,time_since_first_docket_minutes,update_within_2_hours_flag,batch_date,batch_date_updated,same_batch_date_flag,same_batch_date_greater_than_2_hours_flag,update_within_20_minutes_flag
522,2025-01-23 13:51:18.000,London-Concrete--41234494,London-Concrete,WU64ATY,BWL,2025-01-23 13:50:22.528000+00:00,7.5,ALDERMANBURY SQUARE - EC2,S4,160.0,...,1.0,2025-01-27 08:40:22.000,2025-01-27 14:00:00+00:00,5769.624533,False,2025-01-23,2025-01-27,False,False,False
783,2025-01-30 13:12:20.000,London-Concrete--41240808,London-Concrete,EK19VBC,BWL,2025-01-30 13:11:48.693000+00:00,6.0,"1 NORTH QUAY, ASPEN WAY",F5,NaN,...,1.0,2025-02-03 12:21:18.000,2025-02-03 13:08:00+00:00,5756.188450,False,2025-01-30,2025-02-03,False,False,False
258,2025-01-17 16:44:21.000,London-Concrete--41228682,London-Concrete,KX16FOK,BWL,2025-01-17 16:43:42.987000+00:00,7.5,"1 NORTH QUAY, ASPEN WAY",S4,160.0,...,1.0,2025-01-20 07:41:22.000,2025-01-20 16:55:00+00:00,4331.283550,False,2025-01-17,2025-01-20,False,False,False
1237,2025-02-07 13:46:19.000,London-Concrete--41249586,London-Concrete,EK19VBB,BWL,2025-02-07 13:45:48.450000+00:00,5.0,WALTHAM FOREST TOWN HALL - E17,S3,100.0,...,1.0,2025-02-10 15:08:23.000,2025-02-10 13:56:00+00:00,4330.192500,False,2025-02-07,2025-02-10,False,False,False
257,2025-01-17 16:37:20.000,London-Concrete--41228681,London-Concrete,KX16FOF,BWL,2025-01-17 16:37:12.409000+00:00,7.5,"1 NORTH QUAY, ASPEN WAY",S4,160.0,...,1.0,2025-01-20 07:41:20.000,2025-01-20 16:47:00+00:00,4329.793183,False,2025-01-17,2025-01-20,False,False,False
243,2025-01-17 14:27:19.000,London-Concrete--41228564,London-Concrete,RE67BVT,HNL,2025-01-17 14:26:56.642000+00:00,7.5,ST ANN'S ROAD - N15,S4,160.0,...,1.0,2025-01-20 07:43:20.000,2025-01-20 14:29:00+00:00,4322.055967,False,2025-01-17,2025-01-20,False,False,False
842,2025-01-31 15:20:18.000,London-Concrete--41242680,London-Concrete,KT15AAZ,HNL,2025-01-31 15:20:16.918000+00:00,7.5,LORDSHIP LANE - N22,S4,160.0,...,1.0,2025-02-03 07:14:20.000,2025-02-03 15:22:00+00:00,4321.718033,False,2025-01-31,2025-02-03,False,False,False
1251,2025-02-07 15:39:19.000,London-Concrete--41249767,London-Concrete,RX68YCV,OCO,2025-02-07 15:38:58.697000+00:00,6.0,WP17 OLD OAK COMMON LANE - NORTH ACTON,F5,NaN,...,1.0,2025-02-10 08:35:18.000,2025-02-10 15:38:00+00:00,4319.021717,False,2025-02-07,2025-02-10,False,False,False
850,2025-01-31 16:58:22.000,London-Concrete--41242737,London-Concrete,RX68YCV,OCO,2025-01-31 16:58:13.660000+00:00,8.0,OLD OAK COMMON LANE - NORTH ACTON,SF1,NaN,...,1.0,2025-02-03 08:34:19.000,2025-02-03 16:57:00+00:00,4318.772333,False,2025-01-31,2025-02-03,False,False,False
603,2025-01-25 08:19:19.000,London-Concrete--41235976,London-Concrete,RE67BVT,HNL,2025-01-25 08:18:30.458000+00:00,8.0,ADVENT WAY - N18,S2,50.0,...,1.0,2025-01-27 07:21:19.000,2025-01-27 08:19:00+00:00,2880.492367,False,2025-01-25,2025-01-27,False,False,False
